In [1]:
import time
import pandas as pd
import numpy as np
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
from selenium import webdriver
import bs4
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
client_id = 'AJb44uAUgBS5u1gIdz0P'
client_secret = 'ThMXBzW9HG'

url = 'https://openapi.naver.com/v1/search/news?'

header = {"X-Naver-Client-Id":client_id,"X-Naver-Client-secret":client_secret}

symbol = '카카오' # 종목
word_list = ['','주식','뉴스','실적','분석','동향','이미지','기업'] # 검색어
display = 100 # 1~100 고정
start = 1 # 1~1000 고정
sort = 'sim' # sim(정확도), date(시간순)

title_list = []
link_list = []
date_list = []
for word in word_list:
    keyword = symbol + ' ' + word
    start = 1
    for _ in range(10):
        params = {'query':keyword , 'display':display, 'start':start, 'sort':sort}
        req = requests.get(url, params = params, headers = header)
        js = req.json()
        title_list += [js['items'][i]['title'] 
                       for i in range(100) 
                       if 'news.naver.com/main' in js['items'][i]['link'] 
                       if symbol in js['items'][i]['title']]
        link_list += [js['items'][i]['link'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        date_list += [js['items'][i]['pubDate'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        start += 100
    print(f'{keyword} 검색 뉴스기사 크롤링 완료')

카카오  검색 뉴스기사 크롤링 완료
카카오 주식 검색 뉴스기사 크롤링 완료
카카오 뉴스 검색 뉴스기사 크롤링 완료
카카오 실적 검색 뉴스기사 크롤링 완료
카카오 분석 검색 뉴스기사 크롤링 완료
카카오 동향 검색 뉴스기사 크롤링 완료
카카오 이미지 검색 뉴스기사 크롤링 완료
카카오 기업 검색 뉴스기사 크롤링 완료


In [3]:
# date_list 포멧 변경
mon_dict = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}    
ymd_list = []
for date in date_list:
    date_split = date.split()
    y = date_split[3]
    m = mon_dict[date_split[2]]
    d = date_split[1]
    ymd = f'{y}-{m}-{d}'
    ymd_list.append(ymd)

In [11]:
df = pd.DataFrame()
df['ymd'] = ymd_list
df['title'] = title_list
df['link'] = link_list

df['ymd'] = pd.to_datetime(df['ymd']) # ymd열 타입 datetime으로 변경
df = df.drop_duplicates(['link']) # 링크 중복 제거
df = df.drop_duplicates(['title']) # 제목 중복 제거

df = df.sort_values(by=['ymd'], axis=0, ascending = False) # 최신순 정렬
df = df.reset_index(drop=True) # 인덱스 리셋

In [14]:
df.tail(100)

,ymd,title,link
1710,2021-10-05,[단독]<b>카카오</b>VX &quot;스크린골프 철수설? 사실무근&quot;,https://news.naver.com/main/read.naver?mode=LS...
1711,2021-10-01,"[기자수첩] <b>카카오</b>, 다음 상생안 카드가 중요하다",https://news.naver.com/main/read.naver?mode=LS...
1712,2021-09-27,[주간 클라우드 <b>동향</b>/9월④] <b>카카오</b>도 뛰어든 클라우드 시...,https://news.naver.com/main/read.naver?mode=LS...
1713,2021-08-13,&quot;스마트호출 5000원→2000원&quot;…요금 인상 논란에 백기 든 '...,https://news.naver.com/main/read.naver?mode=LS...
1714,2021-08-04,"[주간투자<b>동향</b>] GS칼텍스, <b>카카오</b>모빌리티에 300억 원 ...",https://news.naver.com/main/read.naver?mode=LS...
...,...,...,...
1805,2015-03-05,[글로벌앱<b>동향</b>] 주간 통계 - <b>카카오</b> 게임으로 돌아온 앵그...,https://news.naver.com/main/read.naver?mode=LS...
1806,2015-01-05,[업종<b>동향</b>_코스닥] 다음<b>카카오</b> 중심으로 인터넷업종 폭등,https://news.naver.com/main/read.naver?mode=LS...
1807,2014-02-10,"[주간SNS<b>동향</b>] <b>카카오</b>, API 공개",https://news.naver.com/main/read.naver?mode=LS...
1808,2013-07-05,[주간SNS<b>동향</b>] <b>카카오</b>톡 가입자 1억명 돌파,https://news.naver.com/main/read.naver?mode=LS...
